In [1]:
import elasticsearch
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pickle
import time
from urllib.parse import urljoin

URL = "https://aclanthology.org/volumes/2022.acl-long/"
page = requests.get(URL)


soup = BeautifulSoup(page.content, "html.parser")

In [2]:
es_host = "http://localhost:9200"
es_user = "elastic"
es_password = "1234"

%system docker compose up "-d"

time.sleep(5.) # give docker time to set up

es = elasticsearch.Elasticsearch(
    hosts=es_host,  # "http://localhost:9200"
    basic_auth=(es_user, es_password)   # credentials for basic authentication
)

time.sleep(5.)

es.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'es-docker-single-cluster', 'cluster_uuid': 'T9rYYv05RRWXv_qoiKOQCg', 'version': {'number': '8.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'a846182fa16b4ebfcc89aa3c11a11fd5adf3de04', 'build_date': '2022-11-17T18:56:17.538630285Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
authors_mapping = {
    "properties": {
        "name": {"type": "keyword"},
        "url": {"type": "keyword"},
        "papers": {"type": "keyword"},
    }
}

setting = {
     "analysis": {
      "analyzer": {
        "english_analyzer": {
            "type": "standard",
            "stopwords": "_english_",
            "tokenizer": "standard"
        }
      }
    }
}
paper_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": {
                    "tokenizer": "standard"
                }
            }
        }
    },
    "mappings":{
        "properties": {
        "id": {"type": "keyword"},
        "title": {"type": "text"},
        "date": {"type": "integer"},
        "abstract": {
            "type": "text",
            "analyzer": "my_analyzer"},
        "authors": {"type": "keyword"}
        }
    }
}
# todo: neue Paper?
# todo: lieber von Konferenz aus denken - histories erfassen, adativ
# todo: crawlerklasse mit spezifischen Unterklassen
# todo: holen was geht, stichwort Datapipeline
# todo: Embedding abspeichern
# todo: Suchfunktion (auf Titel/Abstract) nutze "swelte" (Frontend-Programmierung) - Backend: semantische/lexikografische Suche
# todo: BA: Analysen (auf Autoren, ...), Topic-Modell - Clustering (Bird-Modeling, sentence-bird) - Frontend mit Topic-Suche
# todo: Wissenschaftliche Fragestellung: Verschiedene Topic-Modelle vergleichen, nach Metriken:
# todo: Ground-Truths? - Findet man was, z.B. Topics in TOC
# todo: dogsfarm

In [8]:
#es.options(ignore_status=[400,404]).indices.delete(index='paper')
#es.options(ignore_status=[400,404]).indices.delete(index='author')

#es.indices.create(index="paper", settings=paper_settings["settings"], mappings=paper_settings["mappings"])
#es.indices.create(index="author", mappings=authors_mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'author'})

In [7]:
print(es.cat.indices())

yellow open paper  8AuDesp_S3-JJDFIyEJOfA 1 1 142631 0 105.4mb 105.4mb
yellow open author rfwpGqw7SCuXzbKyhC18Nw 1 1  69077 0  10.3mb  10.3mb



In [4]:
def get_links_people():
    everybody = set()
    for letter in tqdm(list(map(chr, range(97, 123)))):
        link = "https://aclanthology.org/people/" + letter + "/"
        soup = BeautifulSoup(requests.get(link).content, "html.parser")
        everybody = everybody.union(set([link + t["href"] for t in soup.find_all("a", attrs={"href":re.compile(letter + ".*")})]))
    return everybody

In [11]:
#all_persons = get_links_people()
#pickle.dump(all_persons, open("remaining_author_links.pkl", 'wb'))

100%|██████████| 26/26 [00:58<00:00,  2.24s/it]


In [5]:
def get_everything_via_author(author_link, host, auth,
                              index1, index2, all_ids = set()):
    '''
    get an authors link + name + papers (id)
    get papers id, title, abstract, date + author (via link)
    :param author_link:
    :return:
    '''
    try:
        soup = BeautifulSoup(requests.get(author_link).content, "html.parser")
    except:
        time.sleep(3.)
        soup = BeautifulSoup(requests.get(author_link).content, "html.parser")
    author_name = soup.h2.text
    author_link = author_link.split("/")[-3]+"/" + author_link.split("/")[-2]
    paper_ids = [id["href"].split(".bib")[0].split("/")[1] for id in soup.find_all("a", attrs={"title":"Export to BibTeX"})]
    requests.post(   # make POST request
    # set target url -> http://localhost:9200/products/_doc
        urljoin(host, f"{index1}/_doc"),
        auth=(auth[0], auth[1]),
        json={
            "name": author_name,
            "url": author_link,
            "papers": paper_ids,
        },
    )
    paper_ids = set(paper_ids).difference(all_ids)
    all_ids = all_ids.union(paper_ids)
    for p in soup.find_all(lambda tag: tag.name == 'span' and
                                   tag.get('class') == ['d-block']):
        p_id = p.strong.a["href"].split("/")[1]
        if not p_id in paper_ids:
            continue
        p_title = p.strong.a.text
        authors_links = [t["href"].split("/")[-3]+"/"+t["href"].split("/")[-2] for t in p.find_all("a", attrs={"href":re.compile(r"/people.*")})]
        abstract_id = "abstract-"+ "--".join(p_id.split("."))
        try:
            p_abstract = soup.find("div", attrs={"id":f"{abstract_id}"}).div.text
        except:
            p_abstract = None
        year = re.findall(r'\d+', p_id[:4])[0]
        if len(year) == 2:
            year = int(year)
            if year < 45:
                year += 2000
            else:
                year += 1900
        else:
            year = int(year)

        res = requests.post(   # make POST request
        # set target url -> http://localhost:9200/products/_doc
            urljoin(host, f"{index2}/_analyze"),
            auth=(auth[0], auth[1]),
            json={
                "analyzer": "standard",
                "text":p_abstract,
            },
        )
        try:
            p_abstract = [token["token"] for token in res.json()["tokens"]]
        except:
            p_abstract=None

        requests.post(   # make POST request
        # set target url -> http://localhost:9200/products/_doc
            urljoin(host, f"{index2}/_doc"),
            auth=(auth[0], auth[1]),
            json={
                "id": p_id,
                "title": p_title,
                "date": year,
                "abstract": p_abstract,
                "authors":authors_links
            },
        )
    return all_ids


def get_index_author(authors_links, host, auth, indices):
    all_p_ids = set()
    pbar = tqdm(total = len(authors_links)+1)
    counter = 0
    while authors_links:
        link = authors_links.pop()
        all_p_ids = get_everything_via_author(link, host, auth, index1=indices[0],
                                             index2=indices[1], all_ids=all_p_ids)
        if counter >= 10: # Reduce memory accesses
            pickle.dump(authors_links, open("remaining_author_links.pkl", 'wb'))
            counter = 0
        counter += 1
        pbar.update(1)
    pbar.close()
    return None


In [6]:
remaining_persons = pickle.load(open("./remaining_author_links.pkl", 'rb'))
get_index_author(remaining_persons, host=es_host, auth=[es_user, es_password], indices=["author", "paper"])

# Expect roughly 1.25it/s

100%|█████████▉| 42676/42677 [7:25:48<00:00,  1.60it/s]   


In [ ]:
es.search(index="paper")

In [ ]:
letter = "a"
soup = BeautifulSoup(requests.get("https://aclanthology.org/people/a/").content, "html.parser")
[t["href"] for t in soup.find_all("a", attrs={"href":re.compile(letter + ".*")})]

In [ ]:
print([t.text for t in soup.find_all("a", attrs={"href":re.compile(r"/people.*")})])
print(soup.find_all("div", attrs={"class":"card bg-light mb-2 mb-lg-3 collapse abstract-collapse"})[30].div.text)

In [ ]:
print(soup.find_all("div", attrs={"id":re.compile(r"abstract-2022--acl-long--.*")})[0].div.text)

In [8]:
print(soup.find_all("a", attrs={"class":"align-middle", "href":re.compile(r"/2022.acl-long.[0123456789]*/")})[1].text)

AdapLeR: Speeding up Inference by Adaptive Length Reduction


In [9]:
authors = []
for i in range(len(soup.find_all(lambda tag: tag.name == 'span' and
                                   tag.get('class') == ['d-block']))):
    authors.append([t.text for t in soup.find_all(lambda tag: tag.name == 'span' and
                                   tag.get('class') == ['d-block'])[i].find_all("a", attrs={"href":re.compile(r"/people.*")})])

In [11]:
abstracts = []
for t in soup.find_all("div", attrs={"id":re.compile(r"abstract.*")}):
    abstracts.append(t.div.text)

In [13]:
titles = []
for i, t in enumerate(soup.find_all("a", attrs={"class":"align-middle", "href":re.compile(r"/2022.acl-long.[0123456789]*/")})):
    titles.append(t.text)

In [16]:
links = []
for i in range(len(soup.find_all(lambda tag: tag.name == 'span' and
                                   tag.get('class') == ['d-block']))):
    links.append([t["href"] for t in soup.find_all(lambda tag: tag.name == 'span' and
                                   tag.get('class') == ['d-block'])[i].find_all("a", attrs={"href":re.compile(r"/people.*")})])
all_links = [item for sublist in links for item in sublist]

In [107]:
authors_with_links = dict(zip([t.text for t in soup.find_all("a", attrs={"href":re.compile(r"/people.*")})], all_links))